### Poisson Ratings
https://penaltyblog.readthedocs.io/en/latest/ratings/massey_ratings.html

https://www.pro-football-reference.com/years/2022/games.htm


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import penaltyblog as pb

In [3]:
import nfl_data_py as nfl

In [4]:
try:
    import nfl_data_py as nfl
except:
    ! pip install nfl_data_py

In [5]:
semana = 7
temporada = 2022
normalizacion = 1
MAX = 10
df = nfl.import_schedules([temporada])
df = df[df['week']<semana]

In [6]:
df = df.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})


In [7]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,31.0,10.0
6138,27.0,26.0
6139,26.0,24.0
6140,10.0,19.0
6141,23.0,20.0
...,...,...
6226,10.0,24.0
6227,9.0,19.0
6228,24.0,20.0
6229,17.0,26.0


In [8]:
df[df['AwayTeam']=='BUF']

,game_id,season,game_type,week,gameday,weekday,gametime,AwayTeam,FTHG,HomeTeam,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6137,2022_01_BUF_LA,2022,REG,1,2022-09-08,Thursday,20:20,BUF,31.0,LA,...,NaN,00-0034857,00-0026498,Josh Allen,Matthew Stafford,Sean McDermott,Sean McVay,Carl Cheffers,LAX01,SoFi Stadium
6173,2022_03_BUF_MIA,2022,REG,3,2022-09-25,Sunday,13:00,BUF,19.0,MIA,...,NaN,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Alex Kemp,MIA00,Hard Rock Stadium
6188,2022_04_BUF_BAL,2022,REG,4,2022-10-02,Sunday,13:00,BUF,23.0,BAL,...,NaN,00-0034857,00-0034796,Josh Allen,Lamar Jackson,Sean McDermott,John Harbaugh,Jerome Boger,BAL00,M&T Bank Stadium
6228,2022_06_BUF_KC,2022,REG,6,2022-10-16,Sunday,16:25,BUF,24.0,KC,...,NaN,00-0034857,00-0033873,Josh Allen,Patrick Mahomes,Sean McDermott,Andy Reid,Brad Allen,KAN00,GEHA Field at Arrowhead Stadium


In [9]:
if normalizacion == 1:
    df['FTHG'] = (df['FTHG'] - df['FTHG'].min()) / ( df['FTHG'].max() - df['FTHG'].min()) * MAX
    df['FTAG'] = (df['FTAG'] - df['FTAG'].min()) / ( df['FTAG'].max() - df['FTAG'].min()) * MAX

In [10]:
df[["FTHG", "FTAG"]].mean()

FTHG    4.392730
FTAG    4.566869
dtype: float64

In [11]:
df[['FTHG','FTAG']]

,FTHG,FTAG
6137,6.458333,1.666667
6138,5.625000,5.476190
6139,5.416667,5.000000
6140,2.083333,3.809524
6141,4.791667,4.047619
...,...,...
6226,2.083333,5.000000
6227,1.875000,3.809524
6228,5.000000,4.047619
6229,3.541667,5.476190


In [12]:
pr = nfl.import_schedules([temporada])
pr = pr[pr['week']==semana ]
juegos = pr['week'].count()

In [13]:
pr = pr.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})

In [14]:
pois = pb.models.PoissonGoalsModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])
pois.fit()
print('Poisson Model')
for i in range(0,juegos):
    EL = pr.iloc[i]['HomeTeam']
    EV = pr.iloc[i]['AwayTeam']
    probs = pois.predict(EL, EV)
    print(f'{EL} vs {EV}')
    print('Local               - Empate -             Visitante')
    print(probs.home_draw_away)

Poisson Model
ARI vs NO
Local               - Empate -             Visitante
[0.5065041027654202, 0.1360360618526528, 0.3573532344701322]
BAL vs CLE
Local               - Empate -             Visitante
[0.0830620511247249, 0.051866603494791434, 0.8208978104430327]
CAR vs TB
Local               - Empate -             Visitante
[0.7000084683091976, 0.13046560675842278, 0.16951228581115885]
CIN vs ATL
Local               - Empate -             Visitante
[0.24641965284239112, 0.1312441200541079, 0.6222646803976737]
DAL vs DET
Local               - Empate -             Visitante
[0.23305576922549212, 0.10156782738758705, 0.6612827030420646]
JAX vs NYG
Local               - Empate -             Visitante
[0.4222863082471779, 0.14832652415297562, 0.42936814051695965]
TEN vs IND
Local               - Empate -             Visitante
[0.41166681573338026, 0.14613997011221405, 0.4421668862567803]
WAS vs GB
Local               - Empate -             Visitante
[0.4814430904661922, 0.1603958459040489

In [15]:
pois

Module: Penaltyblog

Model: Poisson

Number of parameters: 65
Log Likelihood: -358.726
AIC: 847.452

Team                 Attack               Defence             
------------------------------------------------------------
ARI                  1.045                0.08                
ATL                  1.079                0.412               
BAL                  0.998                0.868               
BUF                  0.269                0.62                
CAR                  1.182                0.082               
CHI                  1.131                -0.009              
CIN                  0.723                0.421               
CLE                  1.338                0.517               
DAL                  0.73                 0.349               
DEN                  0.75                 -0.175              
DET                  1.555                0.831               
GB                   1.094                0.257               
HOU                

In [16]:
pois.get_params()

{'attack_ARI': 1.0453658178094067,
 'attack_ATL': 1.079431187363565,
 'attack_BAL': 0.9984436296079033,
 'attack_BUF': 0.2688094985273027,
 'attack_CAR': 1.182260923713708,
 'attack_CHI': 1.1307872236457814,
 'attack_CIN': 0.7232213994743643,
 'attack_CLE': 1.3377832596220653,
 'attack_DAL': 0.7303150334380958,
 'attack_DEN': 0.7503296652662854,
 'attack_DET': 1.5551114986431522,
 'attack_GB': 1.0935078640261373,
 'attack_HOU': 1.1074404679362109,
 'attack_IND': 0.9478952681616304,
 'attack_JAX': 1.0007955078737478,
 'attack_KC': 1.2948960328829227,
 'attack_LA': 1.0702087622867784,
 'attack_LAC': 1.1292565821928817,
 'attack_LV': 1.2931508919724986,
 'attack_MIA': 1.1068206238799738,
 'attack_MIN': 0.830568773249577,
 'attack_NE': 0.6055834755569847,
 'attack_NO': 1.308089331726023,
 'attack_NYG': 0.8829374289184715,
 'attack_NYJ': 0.8614924824609496,
 'attack_PHI': 0.7857559692981849,
 'attack_PIT': 1.160982514739654,
 'attack_SEA': 1.3130995999825086,
 'attack_SF': 0.594007532382930